#### Check the TensorFlow version and save it
SageMaker's TensorFlow version is 1.15.5. This is the reason why we created a virtual environment to specifically use Python 3.7 + TensorFlow 1.15.5 to build the model.

In [1]:
# Import library
import tensorflow as tf

# Print the version and save it
print(tf.__version__)
tf_framework_version = tf.__version__


1.15.5


#### Import libraries and retrieve the IAM
Retrieve the AWS Identity and Access Management (IAM). Some of the codes are tweaks from the code in <a href ="https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/">this post</a>.

In [2]:
# import libraries
from sagemaker import get_execution_role
from sagemaker import Session

# Retrieve the IAM
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()

#### Download the model
The following code downloads the zip file we uploaded in the S3 bucket.

In [3]:
# Create a directory called keras_model
!mkdir keras_model

# Download the zip file we uploaded
!aws s3 cp s3://spam-detector-s3/ann_model.zip ann_model.zip

# Unzip the file in the keras_model folder
!unzip ann_model.zip -d keras_model

download: s3://spam-detector-s3/ann_model.zip to ./ann_model.zip  
Archive:  ann_model.zip
  inflating: keras_model/ann_model.json  
  inflating: keras_model/ann_model_weights.h5  


#### Load the model
The following <a href= "https://stackoverflow.com/questions/53183865/unknown-initializer-glorotuniform-when-loading-keras-model">post</a> was very helpful.

In [4]:
# Import libraries
import keras
import os
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import model_from_json
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

# Load the model
with open(os.path.join('keras_model', 'ann_model.json'), 'r') as fp:
    loaded_model_json = fp.read()
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        loaded_model = model_from_json(loaded_model_json)

# Load the weights
loaded_model.load_weights('keras_model/ann_model_weights.h5')

Using TensorFlow backend.


#### Export the Keras model to the TensorFlow ProtoBuf format
This will translate the Keras model to a format that SageMaker can read. The output will be saved in the 'export/Servo/1" folder as "saved_model.pb".

In [5]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed 
model_version = '1'
export_dir = 'export/Servo/' + model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

session = tf.compat.v1.Session()
init_op = tf.compat.v1.global_variables_initializer()
session.run(init_op)

# Save the meta graph and variables
builder.add_meta_graph_and_variables(
    sess=session, tags=[tag_constants.SERVING], 
    signature_def_map={"serving_default": signature})
builder.save() 

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


b'export/Servo/1/saved_model.pb'

#### Compress and upload files to S3
We are going to compress the file in the tar.gz format and then upload the file to our S3 bucket. The following <a href = "https://baraaalbourghli.medium.com/deploy-your-keras-or-tensorflow-machine-learning-model-to-aws-using-amazon-sagemaker-how-to-2d88a6e779cc">post</a> was very helpful.

In [7]:
# Import libraries
import tarfile
import sagemaker

# Compress the file
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

# Upload the file to the S3
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

#### Deploy the model 
Prepare for the deployment.

In [8]:
# Import library
from sagemaker.tensorflow.serving import Model

# Instance Type
instance_type = 'ml.c5.xlarge' 

# Model
sm_model = Model(model_data=inputs, 
                 framework_version=tf_framework_version,
                 role=role)

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Implement the deployment.

In [9]:
%%time
uncompiled_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type) 
print('/Model is deployed')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!/Model is deployed
CPU times: user 160 ms, sys: 13.3 ms, total: 174 ms
Wall time: 2min 1s


The model is now deployed!

Let's check the endpoint to double-check. Go to SageMaker Dashboard -> expand inference -> Endpoints. If you see something like the following, then it means that your model has been successfully deployed.
<img src ="https://github.com/youngdataspace/Build-and-Deploy-a-Spam-Detector-Model-on-AWS/blob/main/image_endpoint.png?raw=true">

#### Delete the endpoint
Don't forget to delete the endpoint by running the following code. Otherwise, you may incur charges. 

In [10]:
sagemaker_session.delete_endpoint(uncompiled_predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-west-1:345244648414:endpoint/tensorflow-inference-2022-04-20-23-20-27-076".

## Conclusion
Hopefully, this post helped you understand how to train and evaluate an ANN model and deploy it on AWS using SageMaker. If you have any comments or suggestions, email me at y.s.yoon@berkeley.edu.
I am planning on updating this post to include (1) the demonstration of calling the API and (2) monitoring the model.